In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt

2022-07-26 21:51:10.009876: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
@tf.function
def oscillatorModelOde(t, Y, can): 

    #V = can[12]
    VA = can[12]
    #A = V/VA
    sigma = 0.001
    #y1 = VA/(2*sigma)
    
    
    ka1 = can[0]
    kb1 = can[1]
    kcat1 = can[2]

    ka2 = can[3]
    kb2 = can[4]

    ka3 = can[5]
    kb3 = can[6]

    ka4 = can[7]
    kb4 = can[8]

    ka7 = can[9]
    kb7 = can[10]
    kcat7 = can[11]

    ka5 = can[9]
    kb5 = can[10]
    kcat5 = can[11]

    ka6 = can[0]
    kb6 = can[1]
    kcat6 = can[2]


    # ka6 = can[12] 
    # kb6 = can[13]
    # kcat6 = can[14]

    
    #initial conditions
    L = Y[0]   # make sure in uM
    Lp = Y[1]
    K = Y[2]
    P = Y[3]
    LK = Y[4] 
    A = Y[5]
    LpA = Y[6] 
    LpAK = Y[7] 
    LpAP = Y[8] 
    LpAPLp = Y[9] 
    LpAKL = Y[10] 
    LpP = Y[11] 

    dL = (kb1*LK) - (ka1*L*K) + (kcat5*LpAPLp) + (kb6*LpAKL) - ((VA/(2*sigma))*ka6*LpAK*L) + (kcat7*LpP)
    dLp = (kcat1*LK) + (kb2*LpA) - (ka2*Lp*A) + (kb5*LpAPLp) - ((VA/(2*sigma))*ka5*Lp*LpAP) + (kcat6*LpAKL) - (ka7*Lp*P) + (kb7*LpP)
    dK = (kb1*LK) - (ka1*L*K) + (kcat1*LK) + (kb3*LpAK) - (ka3*LpA*K)
    dP = (kb4*LpAP) - (ka4*LpA*P) - (ka7*Lp*P) + (kb7*LpP) + (kcat7*LpP)
    dLK = (ka1*L*K) - (kb1*LK) - (kcat1*LK)
    dA = (kb2*LpA) - (ka2*Lp*A)
    dLpA = (ka2*Lp*A) - (kb2*LpA) + (kb3*LpAK) - (ka3*LpA*K) + (kb4*LpAP) - (ka4*LpA*P)
    dLpAK = (ka3*LpA*K) - (kb3*LpAK) + (kb6*LpAKL) - ((VA/(2*sigma))*ka6*LpAK*L) + (kcat6*LpAKL)
    dLpAP = (ka4*LpA*P) - (kb4*LpAP) + (kb5*LpAPLp) - ((VA/(2*sigma))*ka5*LpAP*Lp) + (kcat5*LpAPLp)
    dLpAPLp = ((VA/(2*sigma))*ka5*LpAP*Lp) - (kb5*LpAPLp) - (kcat5*LpAPLp)
    dLpAKL = ((VA/(2*sigma))*ka6*LpAK*L) - (kb6*LpAKL) - (kcat6*LpAKL)
    dLpP = (ka7*Lp*P) - (kb7*LpP) - (kcat7*LpP)

    return([dL, dLp, dK, dP, dLK, dA, dLpA, dLpAK, dLpAP, dLpAPLp, dLpAKL, dLpP])	

In [3]:
t = np.linspace(0,100,num=10000)

candidate = [0.5399439956871153,
 32.49731125365913,
 459.0713154241908,
 0.6703697205723796,
 98.55386776137301,
 66.94761309866233,
 187.88476784879947,
 0.732829082837601,
 0.5932803922080772,
 0.8833353263653272,
 79.8349667706061,
 98.51067194455857,
 1.21704450980531,
 8.973816043747753,
 0.6170991018130821,
 0.7934153177539267,
 2.6856696379362606]

initials = [tf.constant(candidate[13]),tf.constant(0.0),tf.constant(candidate[14]),
            tf.constant(candidate[15]),tf.constant(0.0),tf.constant(candidate[16]),tf.constant(0.0),tf.constant(0.0),
            tf.constant(0.0),tf.constant(0.0),tf.constant(0.0),tf.constant(0.0)]

2022-07-26 21:51:18.936760: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-26 21:51:19.486381: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2379 MB memory:  -> device: 0, name: Quadro P1000, pci bus id: 0000:65:00.0, compute capability: 6.1


In [4]:
initials = tf.constant([candidate[13],0.0,candidate[14],candidate[15],0.0,candidate[16],0.0,0.0,0.0,0.0,0.0,0.0])
candidate = tf.constant(candidate)


In [5]:
t_begin = tf.constant(0.0)
t_end = tf.constant(100.0)

In [6]:
sol = tfp.math.ode.BDF().solve(oscillatorModelOde, t_begin, initials, solution_times=tfp.math.ode.ChosenBySolver(t_end),constants={'can':candidate})

2022-07-26 21:51:30.290782: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x562b0b77b8c0
